In [1]:
# Install dependencies and libraries
!pip install -r requirements.txt

You should consider upgrading via the 'c:\users\pinayourvishal\local workspace\asset_track\virtualenv\scripts\python.exe -m pip install --upgrade pip' command.



  Using cached xlwings-0.24.9.tar.gz (789 kB)
  Using cached segno-1.3.3-py2.py3-none-any.whl (81 kB)
  Using cached pandas-1.3.3-cp39-cp39-win_amd64.whl (10.2 MB)
  Using cached numpy-1.21.2-cp39-cp39-win_amd64.whl (14.0 MB)
  Using cached openpyxl-3.0.9-py2.py3-none-any.whl (242 kB)
  Using cached Pillow-8.3.2-cp39-cp39-win_amd64.whl (3.2 MB)
  Using cached python_barcode-0.13.1-py3-none-any.whl (217 kB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Using legacy 'setup.py install' for xlwings, since package 'wheel' is not installed.
    Running setup.py install for xlwings: started
    Running setup.py install for xlwings: finished with status 'done'


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import xlwings as xw
import tempfile
import datetime
# For Barcode generation
from barcode import EAN13
# import ImageWriter to generate an image file
from barcode.writer import ImageWriter
# For QR Code generation
import segno

# List the Excel Dependencies

In [4]:
# TODO: Use Selenium to get dump file automatically
airwatch_dump = "./src/Grid_Report_export.csv"
invoices_list = "./src/Invoices.xlsx"
offline_peripherals_list = "./src/Peripherals.xlsx"

## Get the Workspace One Data Dump


In [5]:
df_dump = pd.read_csv(airwatch_dump)
# Summary information
df_dump.info()
df_dump.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Last Seen           105 non-null    object 
 1   Friendly Name       105 non-null    object 
 2   Ownership           105 non-null    object 
 3   Username            94 non-null     object 
 4   First Name          94 non-null     object 
 5   Last Name           94 non-null     object 
 6   Email               94 non-null     object 
 7   Platform            105 non-null    object 
 8   OS                  105 non-null    object 
 9   Model               105 non-null    object 
 10  Phone               0 non-null      float64
 11  Tags                94 non-null     object 
 12  Enrollment          105 non-null    object 
 13  Compliance Status   105 non-null    object 
 14  Display Name        94 non-null     object 
 15  Management          105 non-null    object 
 16  Asset Nu

,Last Seen,Friendly Name,Ownership,Username,First Name,Last Name,Email,Platform,OS,Model,...,Wi-Fi IP Address,Public IP Address,Wi-Fi MAC Address,Notes,WNS Status,DM Last Seen,Build Version,Wi-Fi SSID,Android Management,CPU Architecture
0,9/29/2021 9:45:24 AM,CNHKG03-WIW018,C,Michelle.Li,Michelle,Li,Michelle.Li@hk.rhenus.com,WinRT,10.0.19043,OptiPlex 3070,...,NaN,210.3.202.67,NaN,False,True,9/29/2021 9:24:55 AM,1237,NaN,NaN,X64
1,9/29/2021 9:43:32 AM,CNHKG01-WIL069,C,Echo.Chan,Echo,Chan,Echo.Chan@hk.rhenus.com,WinRT,10.0.19043,81B0,...,NaN,180.161.15.6,94:B8:6D:3B:6D:7A,False,True,9/29/2021 9:31:57 AM,1237,NaN,NaN,X64
2,9/29/2021 9:43:25 AM,CNHKG01-WIL037,C,ivy.ip,Ivy,Ip,ivy.ip@hk.rhenus.com,WinRT,10.0.19043,Latitude 7480,...,NaN,118.140.216.98,74:E5:F9:F0:BD:BF,False,True,9/29/2021 9:12:26 AM,1237,NaN,NaN,X64
3,9/29/2021 9:40:47 AM,CNHKG01-WIL056,C,Chindy.Lui,Chindy,Lui,Chindy.Lui@hk.rhenus.com,WinRT,10.0.19043,81B0,...,NaN,118.140.216.98,B8:08:CF:E7:96:81,False,True,9/29/2021 6:05:03 AM,1237,NaN,NaN,X64
4,9/29/2021 9:40:42 AM,CNHKG04-WIW006,C,TszWah.Ho,TszWah,Ho,TszWah.Ho@hk.rhenus.com,WinRT,10.0.19043,OptiPlex 3070,...,NaN,203.198.186.172,60:F2:62:9A:43:F4,False,True,9/29/2021 6:00:02 AM,1237,NaN,NaN,X64
5,9/29/2021 9:40:31 AM,CNHKG01-WIW059,C,jackie.lam,Jackie,Lam,jackie.lam@hk.rhenus.com,WinRT,10.0.19043,OptiPlex 3050,...,NaN,118.140.216.98,NaN,False,True,9/29/2021 2:06:03 AM,1237,NaN,NaN,X64
6,9/29/2021 9:39:51 AM,CNHKG04-WIL001,C,Will.Wong,Will,Wong,Will.Wong@hk.rhenus.com,WinRT,10.0.19043,Latitude 5400,...,NaN,203.198.186.172,5C:80:B6:3C:F2:67,False,True,9/29/2021 9:39:51 AM,1237,NaN,NaN,X64
7,9/29/2021 9:38:27 AM,CNHKG01-WIL036,C,Kenny.Lau,Kenny,Lau,Kenny.Lau@hk.rhenus.com,WinRT,10.0.19043,Latitude 7480,...,NaN,118.140.216.98,40:A3:CC:68:0B:B9,False,True,9/29/2021 9:06:16 AM,1237,NaN,NaN,X64
8,9/29/2021 9:37:16 AM,CNHKG04-WIW004,C,Ming.Lee,Ming,Lee,Ming.Lee@hk.rhenus.com,WinRT,10.0.19043,OptiPlex 3070,...,NaN,203.198.186.172,60:F2:62:99:E8:7D,False,True,9/29/2021 8:05:02 AM,1237,NaN,NaN,X64
9,9/29/2021 9:36:30 AM,CNHKG01-WIW058,C,Leo.Yeung,Leo,Yeung,Leo.Yeung@hk.rhenus.com,WinRT,10.0.19043,OptiPlex 3050,...,NaN,118.140.216.98,NaN,False,True,9/29/2021 4:15:05 AM,1237,NaN,NaN,X64


# TODO: GET LOCAL PERIPHERALS DATA

# Get Invoices

In [6]:
# The row to start considering data from
header = 3
# Read Invoices list
df_invoice = pd.read_excel(invoices_list, header=header)
# Show summary information
df_invoice.info()
df_invoice.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Invoice Number            23 non-null     object        
 1   Serial Number             251 non-null    object        
 2   Purchase Date             23 non-null     datetime64[ns]
 3   Warranty Expiration Date  23 non-null     datetime64[ns]
 4   Invoice File              23 non-null     object        
dtypes: datetime64[ns](2), object(3)
memory usage: 9.9+ KB


,Invoice Number,Serial Number,Purchase Date,Warranty Expiration Date,Invoice File
0,1400666873,6TF31L2,2017-08-20,2022-08-20,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
1,1400666873,6TDW0L2,2017-08-20,2022-08-20,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
2,1400614031,F17XPC2,2016-10-05,2021-10-05,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
3,2017-4159,PF0Q6AFJ,2017-04-21,2020-04-21,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
4,1400666873,6TB41L2,2017-08-20,2022-08-20,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
5,1400598945,571MJC2,2016-07-12,2019-07-13,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
6,1400807900,D57D3X2,2020-01-03,2023-01-03,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
7,1400813049,1MRLQ13,2020-02-13,2023-02-13,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
8,1400813049,305DQ13,2020-02-13,2023-02-13,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
9,1400807900,7S6D3X2,2020-01-03,2023-01-03,https://rhenusasia.sharepoint.com/:b:/s/ITMast...


# TODO: Clean Dump to reflect MasterData template

In [7]:
# Specify columns to be used for local records and master data, in the right order.

# devices_columns = {"Status", "Email", "City Code", "Computer Name", "Manufacturer", "Category", "Serial Number", "Friendly Name", "Model"}
# df_device = df_dump[devices_columns]

# Merge invoice information with Workspace One dump

In [8]:
# List important columns
## TODO: Generalise for MasterData
devices_columns = {"Serial Number", "Friendly Name", "Email", "Model", "Tags"}
df_device = df_dump[devices_columns]
# Join invoice information and device information
df_device_invoice = pd.merge(df_device, df_invoice, on="Serial Number", how="left")
# Drop duplicated entries
df_device_invoice = df_device_invoice.drop_duplicates()

# Ensure validity of cleaned dataset

In [9]:
# Check if data cleaning has eliminated any serial numbers
init_serials = df_device["Serial Number"].to_list()
final_serials = df_device_invoice["Serial Number"].to_list()
init_serials = set(init_serials)
final_serials = set(final_serials)
missing = list(sorted(final_serials - init_serials))
added = list(sorted(final_serials - init_serials))
print('Missing Serial Numbers: ', missing)
print('Added Serial Numbers: ', added)

Missing Serial Numbers:  []
Added Serial Numbers:  []


# Data formatting

In [12]:
# Add columns
## TODO: auto-generate location and location 
df_device_invoice["QR Code"] = ""
df_device_invoice["Location"] = ""
df_device_invoice["Asset Type"] = ""
# Re-arrange columns
custom_column_order = ["Location", "Asset Type", "QR Code", "Serial Number", "Friendly Name", "Model", "Tags", "Invoice Number", "Purchase Date", "Warranty Expiration Date", "Invoice File"]
df_device_invoice = df_device_invoice[custom_column_order]
# Summary information
# df_device_invoice.info()
# df_device_invoice.head(100)
# df_device_invoice["Serial Number"].value_counts()


# Export Device-Invoice data to Excel Workbook.

In [13]:
# Generate timestamp
current_day = str(datetime.datetime.now().year) + "_" + str(datetime.datetime.now().month) + "_" + str(datetime.datetime.now().day)
# Varible for book name
book_url = "./output/CNHKG0X_IT_ASSETS_" + current_day + ".xlsx"

df_device_invoice.to_excel(book_url, index=False)

# Generate QR Codes and Bar Codes for all devices.

In [14]:
# Update this with the name of your workbook
book = xw.Book(book_url)
sheet = book.sheets[0]
# Update this with the starting cell of your URLs
start_cell = sheet['D2']
# Get Serial numbers of devices
serial_numbers = start_cell.options(expand='down', ndim=1).value

url_prefix = "https://cn1380.awmdm.com/AirWatch/#/AirWatch/Search?query="

urls = [url_prefix + item for item in serial_numbers]

In [15]:
# # # Loop through each URL and generate the QR code
for id, url in enumerate(urls):
    # Generate the QR code
    qr = segno.make(url)

    with tempfile.TemporaryDirectory() as td:
        # Save the QR code as a temporary svg file. If you are on macOS, use pdf
        # instead and if you don't have Microsoft 365, you may have to use png
        td = str(td)
        filepath = f'{td}/qr.svg'
        qr.save(filepath, scale=5, border=0, finder_dark='#000000')
        # Insert the QR code to the right of the URL
        destination_cell = start_cell.offset(row_offset=id, column_offset=-1)
        # Add temporary image to spreadsheet
        sheet.pictures.add(filepath, left=destination_cell.left, top=destination_cell.top, width=50, height=50)

# Autofit row and column dimensions
sheet.autofit("c")
sheet.autofit("r")

# # Resize first column to make space for QR Code
# sheet.range("A").column_width = 60
# sheet.range("A").row_height = 60